In [3]:
# Add this code to a new cell in your 03-tuning-function-fitting.ipynb notebook

# --- Phase 3: Tuning Function Fitting ---
# Goal: Load the outputs from the previous phases and use them to estimate
# the spatio-temporal receptive field (STRF) for each neuron.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils as U # Assuming your utility functions are in utils.py
import importlib
importlib.reload(U) # Use this if you've made changes to utils.py
import os
import glob

print("Starting Phase 3: Tuning Function Fitting...")

# --- Step 1: Load Inputs from Previous Notebooks ---
# We need the outputs from both preprocessing and spike inference.

try:
    # --- Load Stimulus Data (from 00-background.ipynb) ---
    # This data is required for the fitting process.
    stim_data_path = './data/preprocessed/deconv_and_sta.npz'
    stim_data = np.load(stim_data_path, allow_pickle=True)
    stim_filtered = stim_data['stim_filtered']
    stim_table_filtered_df = pd.DataFrame(stim_data['stim_table_filtered'], columns=stim_data['stim_table_columns'])
    print(f"Successfully loaded stimulus data from: {stim_data_path}")

    # --- Load Spike Inference Data (from 02-spike-inference.ipynb) ---
    # The output file from the spike inference notebook is timestamped,
    # so we find the most recent one in the directory.
    spike_inference_dir = 'data/spike_inference/'
    
    # Find all .npz files in the directory
    list_of_files = glob.glob(os.path.join(spike_inference_dir, '*.npz'))
    if not list_of_files:
        raise FileNotFoundError(f"No spike inference output files found in '{spike_inference_dir}'")
    
    # Get the most recent file based on creation time
    latest_spike_file = max(list_of_files, key=os.path.getctime)
    
    print(f"Loading latest spike inference data from: {latest_spike_file}")
    spike_data = np.load(latest_spike_file)
    inferred_spikes = spike_data['inferred_spikes']
    t_filtered = spike_data['time_vector'] # Use the correct key 'time_vector'
    

except FileNotFoundError as e:
    print(f"ERROR: Could not find a necessary input file: {e}")
    print("Please ensure you have run the previous notebooks (00 and 02) and saved their outputs correctly.")
    # Stop execution if files are missing
    raise

# --- Step 2: Run the Receptive Field Fitting Pipeline ---
# This section calls the functions from your utils.py file in sequence.

# 2a. Bin the inferred spikes to align with stimulus frames
binned_spikes = U.bin_spikes_to_frames(inferred_spikes, t_filtered, stim_table_filtered_df)

# 2b. Prepare the stimulus matrix by flattening the frames
flattened_stim, stim_h, stim_w = U.prepare_stimulus_matrix(stim_filtered)

# 2c. Define the time lags and fit the STRF for all neurons
lags_to_test = [0, 1, 2, 3, 4]
all_rfs_spatiotemporal = U.fit_all_neurons_rfs(binned_spikes, flattened_stim, lags_to_test)

# 2d. Extract the primary spatial component from each STRF using SVD
all_rfs_spatial = U.extract_spatial_rfs_svd(all_rfs_spatiotemporal, stim_h, stim_w)

print("\nReceptive field fitting and analysis complete!")


# --- Step 3: Verification Plot ---
# Visualize the results for one neuron to confirm the pipeline worked.
neuron_to_plot = 10 # You can change this to inspect different neurons

U.visualize_neuron_rf(
    neuron_id=neuron_to_plot,
    spatiotemporal_rf=all_rfs_spatiotemporal[neuron_to_plot],
    spatial_rf=all_rfs_spatial[neuron_to_plot],
    delta=lags_to_test
)

# --- Step 4: Save the output of this notebook ---
# Saving the results is crucial so you don't have to re-run this computationally
# intensive step for the final analysis in the next notebook.
output_path = 'data/preprocessed/rf_fitting_output.npz'
np.savez_compressed(
    output_path,
    all_rfs_spatiotemporal=np.array(all_rfs_spatiotemporal),
    all_rfs_spatial=np.array(all_rfs_spatial)
)
print(f"\nSuccessfully saved receptive field results to: {output_path}")



Starting Phase 3: Tuning Function Fitting...


KeyError: 'stim_filtered is not a file in the archive'